<a href="https://colab.research.google.com/github/talkin24/jaxflax_lab/blob/main/Processing_the_entire_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

효율성을 위해 여러 개의 샘플이 포함된 배치를 구성하여 병렬로 처리함. 특히 모델을 평가할 때는 모든 샘플을 처리하고 마지막에 완전한 배치를 구성하지 않는 나머지 샘플이 손실되지 않도록 하는 것이 중요함.

# The problem

단일 디바이스에서 평가할 때는 마지막 불완전한 배치를 삭제하거나 이전 배치와 다른 모양으로 마지막 배치를 형성할 수 있음. 후자의 경우 XLA가 shape 다형성이 아니기 때문에 eval_step()을 다시 컴파일해야 한다는 단점이 있음

In [ ]:
import collections
import tensorflow_datasets as tfds

collections.Counter(
    tuple(batch['image'].shape)
    for batch in tfds.load('mnist', split='test').batch(per_device_batch_size)
)
# output:
# Counter({(272, 28, 28, 1): 1, (512, 28, 28, 1): 19})

이 문제는 데이터 병렬 처리를 위해 여러 장치를 사용할 때 더욱 두드러짐. 배치 크기를 디바이스 수로 나눌 수 없는 경우 마지막 단계는 단일 디바이스(또는 디바이스의 하위 집합)에서 실행해야 함. 일반적으로 마지막 배치를 삭제하지만 잘못된 결과를 초래할 수 있음.

In [ ]:
sum(
    np.prod(batch['label'].shape)
    for batch in tfds.load('mnist', split='test')
        .batch(per_device_batch_size, drop_remainder=True)
        .batch(jax.local_device_count())
)
# output:
# 9728

여러 호스트를 사용하면 JAX가 SPMD 패러다임을 사용하고 모든 호스트가 동일한 프로그램을 실행해야 하기 때문에 상황이 더욱 복잡해짐. 일반적으로 tfds.split_for_jax_process()를 사용하여 서로 다른 호스트에 대해 겹치지 않는 분할을 형성하지만, 이렇게 하면 호스트마다 다른 숫자가 발생하여 모든 샘플을 처리해야 할 때 서로 다른 JAX 프로그램이 생성될 수 있음

In [ ]:
process_count = 6
[
    len(tfds.load(dataset_name, split=tfds.split_for_jax_process(
        'test', process_index=process_index, process_count=process_count)))
    for process_index in range(process_count)
]
# output:
# [1667, 1667, 1667, 1667, 1666, 1666]

# The solution: padding

서로 다른 호스트에서 서로 다른 장치에서 실행되는 배치의 수를 조정하여 이 문제를 해결할 수 있지만, 이러한 솔루션은 빠르게 복잡해지고 번거로운 로직으로 인해 메인 평가 루프를 읽기 어렵게 만듦

이 문제에 대한 보다 간단한 해결책은 데이터 세트의 끝에 패딩을 사용하여 마지막 배치가 이전 배치와 동일한 크기를 갖도록 하는 것임


## 수동 조정

마지막 배치에는 이전 배치와 동일한 수의 예제가 포함되도록 수동으로 패딩됨. 패딩된 예제에 대한 예측은 계산에서 삭제됨

In [ ]:
shard = lambda x: einops.rearrange(
    x, '(d b) ... -> d b ...', d=jax.local_device_count())
unshard = lambda x: einops.rearrange(x, 'd b ... -> (d b) ...')

correct = total = 0
for batch in ds.as_numpy_iterator():
  images = batch['image']
  n = len(images)
  padding = np.zeros([per_host_batch_size - n, *images.shape[1:]], images.dtype)
  padded_images = np.concatenate([images, padding])
  preds = unshard(get_preds(variables, shard(padded_images)))[:n]
  total += n
  correct += (batch['label'] == preds.argmax(axis=-1)).sum()

## Using `pad_shard_unpad()`

위의 패턴, 즉 패드→샤드→예측→언샤드→언패드 시퀀스를 유틸리티 래퍼 pad_shard_unpad()로 추출하면 위의 평가 루프를 크게 간소화할 수 있음

In [ ]:
correct = total = 0
for batch in ds.as_numpy_iterator():
  preds = flax.jax_utils.pad_shard_unpad(get_preds)(
      vs, batch['image'], min_device_batch=per_device_batch_size)
  total += len(batch['image'])
  correct += (batch['label'] == preds.argmax(axis=-1)).sum()

## Computing metrics in `eval_step()`


메인 평가 루프에서 예측을 반환하고 메트릭을 계산하는 대신, 특히 clu.metrics 또는 clu.metrics와 같은 라이브러리를 사용할 때 메트릭 계산을 평가 단계의 일부로 만들고자 하는 경우가 많음.

이 경우 메트릭을 static_argnums로 전달하고(즉, 샤딩/패딩하지 않음) 반환 값도 static_return으로 처리하고(즉, 샤딩 해제 또는 패딩 해제 없음) 싶을것임

In [ ]:
def eval_step(metrics, variables, batch):
  print('retrigger compilation', {k: v.shape for k, v in batch.items()})
  preds = model.apply(variables, batch['image'])
  correct = (batch['mask'] & (batch['label'] == preds.argmax(axis=-1))).sum()
  total = batch['mask'].sum()
  return dict(
      correct=metrics['correct'] + jax.lax.psum(correct, axis_name='batch'),
      total=metrics['total'] + jax.lax.psum(total, axis_name='batch'),
  )

eval_step = jax.pmap(eval_step, axis_name='batch')
eval_step = flax.jax_utils.pad_shard_unpad(
    eval_step, static_argnums=(0, 1), static_return=True)

# Adding “infinite padding”

위의 솔루션은 대부분의 경우 작동하지만 몇 가지 제한 사항이 있음.

1. 드물게 여러 호스트에서 데이터 세트를 분할해도 배치 수가 달라지는 경우가 있음. n=4097개의 예제로 구성된 데이터 세트가 있고, 이를 각각 d=8개의 로컬 디바이스가 있는 h=8에서 평가하고, 온디바이스 배치 크기가 b=128이라고 가정. 데이터 세트를 균등하게 분할하면 첫 번째 호스트는 4096/8+1==513개의 예제를 얻게 되고 다른 모든 호스트는 4096/8==512개의 예제를 얻게 됩니다. 호스트별로 d*b==512의 배치를 형성하면 첫 번째 호스트에는 두 개의 배치가, 다른 모든 호스트에는 하나의 배치가 생성되어 SPMD 원칙을 위반하고 마지막 psum() 지시문에서 다중 호스트 설정이 중단됩니다(첫 번째 호스트에서만 실행되고 다른 호스트에는 실행되지 않음).

2. ds.filter()를 사용하여 예제를 동적으로 삭제할 때.

이러한 더 복잡한 경우에는 데이터 세트에 "무한 패딩"을 각 호스트에 독립적으로 추가하고 모든 호스트에 패딩되지 않은 예제가 다 떨어질 때까지 예제를 계속 처리할 수 있습니다.

In [ ]:
correct = total = 0
for batch in ds.as_numpy_iterator():
  n = count_p(batch['mask'])[0].item()  # adds sync barrier
  if not n: break

  preds = get_preds(vs, batch['image']).argmax(axis=-1)
  total += n
  correct += count_correct_p(batch['label'], preds, batch['mask'])[0]

SPMD와 MPMD
: 대부분의 병렬 프로그램은 SPMD(Single Program Multiple Data) 형태를 가진다. SPMD 형태 의 병렬프로그램이란 병렬 프로그램을 수행하는 모든 프로세스 또는 스레드가 동일한 하나의 프로그램을 실행하면서 프로그램 내의 함수를 서로 다른 데이터를 가지는 병렬로 실행하거나(도메인 분해) 각각 서로 다른 함수를 맡아 병렬로 실행하는 것을 (기능적 분해) 말한다.
MPMD(Multiple Program Multiple Data) 형태의 병렬 프로그램은 여러 개의 프로그램으로 구성된다. 각 프로세서 또는 스레드가 각각 서로 다른 프로그램을 실행시키면서 프로그램이 필요로 하는 데이터를 통신을 통해 주고 받게 된다.